In [8]:
import requests as req
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [12]:
#교보 데이터 수집

options = Options()
# options.add_argument('--headless')
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=options)
driver.get("https://store.kyobobook.co.kr/bestseller/total/monthly?page=1&per=50")
time.sleep(3)

ol = driver.find_element(By.CSS_SELECTOR, "ol.grid.border-t.border-gray-400.grid-cols-1.pt-9")
li10_kb = ol.find_elements(By.XPATH, '/html/body/div[1]/main/section/div/div/section/ol[1]/li')
li40_kb = ol.find_elements(By.XPATH, '/html/body/div[1]/main/section/div/div/section/ol[2]/li')
li_kb = li10_kb + li40_kb

links_kb = []
titles_kb = []
authors_kb = []
genres_kb = []
for i in range(0,30):
#     title = li[i].find_element(By.CSS_SELECTOR, "a.font-weight-medium").text
    link_kb = li_kb[i].find_element(By.CSS_SELECTOR, "a.font-weight-medium").get_attribute("href")
    links_kb.append(link_kb)

for j in links_kb:
    driver.get(j)
    time.sleep(2)
    title_kb = driver.find_element(By.CSS_SELECTOR, "span.prod_title").text
    author_kb = driver.find_element(By.CSS_SELECTOR, ".author > a").text
    genre_kb = driver.find_elements(By.CSS_SELECTOR, '.btn_sub_depth')[1].text
    titles_kb.append(title_kb)
    authors_kb.append(author_kb)
    genres_kb.append(genre_kb)
driver.quit()

# print(titles_kb)
# print(authors_kb)
# print(genres_kb)

for i in range(len(genres_kb)):
    if genres_kb[i] == '소설':
        genres_kb[i] = '소설/시/희곡/에세이'
    elif genres_kb[i] == '시/에세이':
        genres_kb[i] = '소설/시/희곡/에세이'
print(genres_kb)

bookData_kb = []
for k in range(0,30):
    bookData_kb.append({
            "순위": k+1,
            "제목": titles_kb[k],
            "작가": authors_kb[k],
            "장르": genres_kb[k]
    })
print(bookData_kb)

books_kb_df = pd.DataFrame(bookData_kb)
books_kb_df

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"ol.grid.border-t.border-gray-400.grid-cols-1.pt-9"}
  (Session info: chrome=131.0.6778.109); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6DA8A6CC5+28821]
	(No symbol) [0x00007FF6DA813850]
	(No symbol) [0x00007FF6DA6B578A]
	(No symbol) [0x00007FF6DA7091BE]
	(No symbol) [0x00007FF6DA7094AC]
	(No symbol) [0x00007FF6DA752647]
	(No symbol) [0x00007FF6DA72F33F]
	(No symbol) [0x00007FF6DA74F412]
	(No symbol) [0x00007FF6DA72F0A3]
	(No symbol) [0x00007FF6DA6FA778]
	(No symbol) [0x00007FF6DA6FB8E1]
	GetHandleVerifier [0x00007FF6DABDFCCD+3408029]
	GetHandleVerifier [0x00007FF6DABF743F+3504143]
	GetHandleVerifier [0x00007FF6DABEB61D+3455469]
	GetHandleVerifier [0x00007FF6DA96BDCB+835995]
	(No symbol) [0x00007FF6DA81EB6F]
	(No symbol) [0x00007FF6DA

In [4]:
#yes24 데이터 수집

url = "https://www.yes24.com/Product/Category/MonthWeekBestSeller?categoryNumber=001&pageNumber=1&pageSize=40&type=month&saleYear=2024&saleMonth=11"
res = req.get(url)

bookData_yes24 = []

if res.status_code == 200:
    soup = bs(res.text, "html.parser")
    trs = soup.select("#yesBestList .itemUnit")
    
    rank = 1
    for item in trs:
        if rank > 30:
            break
        title_yes24 = item.select_one(".gd_name").get_text(strip=True)
        href = item.find("a")["href"]
        author_yes24 = item.select('span.authPub.info_auth > a')[0].text.strip()
        
        url2 = "https://www.yes24.com" + href
        res2 = req.get(url2)    
        if res2.status_code == 200:
            book_soup = bs(res2.text, "html.parser")
            links = book_soup.select(".infoSetCont_wrap > dl > dd > ul > li > a")
            if links:
                genre_yes24 = links[1].get_text(strip=True)
                genre_yes24 = genre_yes24.replace("에세이", "소설/시/희곡")
                genre_yes24 = genre_yes24.replace("소설/시/희곡", "소설/시/희곡/에세이")
                genre_yes24 = genre_yes24.replace("경제 경영", "경제/경영")
        

        bookData_yes24.append({
            "순위": rank,
            "제목": title_yes24,
            "작가": author_yes24,
            "장르": genre_yes24
        })
                   
        rank += 1

books_yes24_df = pd.DataFrame(bookData_yes24)
books_yes24_df

,순위,제목,작가,장르
0,1,소년이 온다,한강,소설/시/희곡/에세이
1,2,채식주의자,한강,소설/시/희곡/에세이
2,3,작별하지 않는다,한강,소설/시/희곡/에세이
3,4,트럼프 2.0 시대,박종훈,경제/경영
4,5,서랍에 저녁을 넣어 두었다,한강,소설/시/희곡/에세이
5,6,흰,한강,소설/시/희곡/에세이
6,7,트렌드 코리아 2025,김난도,경제/경영
7,8,하루 한 장 나의 어휘력을 위한 필사 노트,유선경,인문
8,9,희랍어 시간,한강,소설/시/희곡/에세이
9,10,넥서스,유발 하라리,인문


In [ ]:
##########################################################################################
titles_yes24 = books_yes24_df['제목'].tolist()
list_yes24 = list(set(titles_yes24)-set(titles_kb))
list_kb = list(set(titles_kb)-set(titles_yes24))

list_yes24_df = books_yes24_df[books_yes24_df['장르'].isin['순위','제목','작가','장르']]
list_kb_df = books_kb_df[books_kb_df['장르'].isin['순위','제목','작가','장르']]

genrePer_yes24 = books_yes24_df['장르'].value_counts(normalize=True) * 100
genrePer_kb = books_kb_df['장르'].value_counts(normalize=True) * 100


# print(set(titles_yes24)-set(titles_kb))
# print(len(set(titles_yes24)-set(titles_kb)))
# print(set(titles_kb)-set(titles_yes24))
# print(len(set(titles_kb)-set(titles_yes24)))

# books_kb_df.to_csv('book_kb_df.csv', index = False, encoding='utf-8-sig')
# books_yes24_df.to_csv('books_yes24_df.csv', index=False, encoding='utf-8-sig')

books_yes24_df.set_index("순위", inplace=True)
#########################################################################################

NameError: name 'titles_kb' is not defined